<a href="https://colab.research.google.com/github/hongrui16/Google-Colaboratory-Project/blob/master/yolov5x_wheat_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls
!git clone https://github.com/ultralytics/yolov5 

In [ ]:
!pip install -qr yolov5/requirements.txt  # install dependencies (ignore errors)
%cd yolov5

In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
%cd ..
!ls

In [ ]:
# installing pydrive for dataset extraction from Google Drive
!pip install -U -q PyDrive

# dependencies required for the dataset
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticating
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# dataset for Image Classification from Google Drive Sharerable id=1lpASoMgbZorP3iI6q_AY8HRcDk352yao
dataset_downloaded = drive.CreateFile({'id': '1hWNyeII5qLmBQfOtVldzAuRdr2293ZzN'})
dataset_downloaded.GetContentFile('global-wheat-detection.zip')

In [ ]:
!ls

In [ ]:
!unzip global-wheat-detection.zip

In [ ]:
!mkdir -p wheat_dataset/images/train wheat_dataset/images/validation wheat_dataset/labels/train wheat_dataset/labels/validation


In [ ]:
!pwd
!ls

In [ ]:
import pandas as pd
import ast # for converting string to list

df = pd.read_csv('train.csv')
df['bbox'] = df['bbox'].apply(ast.literal_eval)
df = df.groupby("image_id")["bbox"].apply(list).reset_index(name = "bboxes")


# dividing the data int train test slit
from sklearn import model_selection

df_train , df_valid = model_selection.train_test_split(
    df,
    test_size = 0.1,
    random_state = 42,
    shuffle = True
    )

df_train = df_train.reset_index(drop = True)
df_valid = df_valid.reset_index(drop = True)

from tqdm import tqdm
import numpy as np
import shutil 
import os

output_path = '/content/wheat_dataset'

def process_data(dataframe , data_type = "train"): # data_type is used for selecting the specific folder
  for _, row in tqdm(dataframe.iterrows() , total = len(dataframe)):
    image_name = row['image_id']
    bounding_boxes = row['bboxes']

    yolo_data = []

    for bbox in bounding_boxes:
      x = bbox[0]
      y = bbox[1]
      w = bbox[2]
      h = bbox[3]

      # normalising the co-ordinates
      x_center = x + w / 2
      y_center = y + h / 2
      x_center /= 1024.0 
      y_center /= 1024.0 
      w /= 1024.0
      h /= 1024.0

      yolo_data.append([0,x_center,y_center,w,h])

    yolo_data = np.array(yolo_data)
    
    # saving in txt format
    np.savetxt(
        os.path.join(output_path,f"labels/{data_type}/{image_name}.txt"),
        yolo_data,
        fmt=["%d","%f","%f","%f","%f"]
    )

    # copy the images output_path images folder
    shutil.copyfile(
        os.path.join('/content/train',f"{image_name}.jpg"), # from path
        os.path.join(output_path,f"images/{data_type}/{image_name}.jpg")  # to path
        )
    
    
process_data(df_train , data_type = "train")
process_data(df_valid , data_type = "validation")

In [ ]:
f = open("/content/yolov5/data/wheat.yaml", "w")
f.write("train: /content/wheat_dataset/images/train/\nval: /content/wheat_dataset/images/validation/\nnc: 1\nnames: ['wheat_head']")
f.close()

In [ ]:
!cat /content/yolov5/data/wheat.yaml

In [ ]:
!mkdir /content/yolov5/pre-trained_model

In [ ]:
%cd /content/yolov5/pre-trained_model
# download pre-trained model, yolov5x, https://drive.google.com/file/d/1mM8aZJlWTxOg7BZJvNUMrTnA2AbeCVzS/view?usp=sharing
dataset_downloaded = drive.CreateFile({'id': '1mM8aZJlWTxOg7BZJvNUMrTnA2AbeCVzS'})
dataset_downloaded.GetContentFile('yolov5x.pt')

In [ ]:
!pwd
!ls

In [ ]:
%cd /content/yolov5/
!python train.py --img 1024 --batch 32 --epochs 10 --data data/wheat.yaml --cfg models/yolov5s.yaml --name wm --nosave --cache --resume pre-trained_model/yolov5x.pt